In [ ]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
%%appyter markdown

<center>
    <h1> 
        <div style="font-size: 3rem; font-weight: 500"> <style="height:45px;padding:0 5px;display:inline"/>KEA3 Appyter</div>
    </h1>
    <br>
    <div style="font-size: 1.5rem; font-weight: 300">An appyter interface to the kinase enrichment analysis (KEA) database and enrichment analysis tool</div>
</center>

In [ ]:
import json
import requests
from time import sleep
from tabulate import tabulate
from IPython.display import HTML, display, Image, FileLink
import plotly.graph_objects as go
import kaleido
import os 

In [ ]:
def get_kea3_results(gene_set, query_name):
    ADDLIST_URL = 'https://maayanlab.cloud/kea3/api/enrich/'
    payload = {
        'gene_set': gene_set,
        'query_name': query_name
    }

    response = requests.post(ADDLIST_URL, data=json.dumps(payload))
    if not response.ok:
        raise Exception('Error analyzing gene list')
    sleep(1)

    return json.loads(response.text)

In [ ]:
%%appyter hide_code

{% do SectionField(name ='section', title = 'Perform Kinase Enrichment Analysis on a Gene/Protein List', 
                   subtitle = 'Kinase Enrichment Analysis 3 (KEA3) infers upstream kinases whose putative substrates are ' +
                               'overrepresented in a user-inputted list of genes or differentially phosphorylated proteins. ' +
                               'The KEA3 database contains putative kinase-substrate interactions collected from publicly ' +
                               'available datasets. Upload -OR- input a gene/protein list to retrieve the top associated kinases.', 
                   img = 'KEA3_transparent_logo.png')%}

## Generate bar chart and tables containing top associated kinases for different libraries
Libraries are supersets of kinase substrate sets that are aggregated based on the database from which they are derived.

In [ ]:
%%appyter code_exec

gene_list_file = {{ FileField(
        name = 'Upload gene/protein list',
        label = 'Upload gene/protein list',
        default = '',
        examples = {'sample_gene_list.txt': 'https://raw.githubusercontent.com/MaayanLab/KEA3-Appyter/master/sample_gene_list.txt'},
        section = 'section'
    )}}

gene_list_input = {{ TextField(
        name = 'Input gene/protein list',
        label = 'Input gene/protein list (one per row)',
        default = 'ZNF264 \nTMPO \nISL2 \nMAP3K8 \nEFNB1 \nEIF3C \nOSBPL11 \nABCF1 \nUTRN \nOPRK1 \nTSC1 ' +
                    '\nGAB2 \nRPS3P2 \nDDX3X \nPPP1CA \nNF2 \nRBM3 \nIRAK1 \nKCNH2 \nNPR1 \nMOCOS \nITSN2 ' +
                    '\nMITF \nARAF \nDAPK2 \nEPHB2 \nCACNA1G \nYWHAZ \nGMFB',
        section = 'section',
    )}}


# Number of ranked kinases to display for the tables
num_kinases = {{ IntField(
        name = 'Input number of top kinases to display',
        label = 'Number of top kinases for tables',
        description = 'Input any integer from 1 to 100',
        default = 10,
        min = 1, 
        max = 100,
        section = 'section'
    )}}

gene_input = None

if gene_list_file:
    gene_list_file = open(gene_list_file,'r')
    gene_input = gene_list_file.read()
    gene_list_file.close()
elif gene_list_input:
    gene_input = gene_list_input
else:
    display(HTML('<h2>No kinase list was inputted or uploaded.</h2>'))

if gene_input:

    genes = [x.strip() for x in gene_input.split('\n')]
    results = get_kea3_results(genes, 'Query')
    k_libs_palette = {'BioGRID': 'rgb(196, 8, 8)', 'ChengKSIN': 'rgb(244, 109, 67)', 'ChengPPI': 'rgb(242, 172, 68)',
                      'HIPPIE': 'rgb(236, 252, 68)', 'mentha': 'rgb(165, 242, 162)', 'MINT': 'rgb(92, 217, 78)',
                      'PhosDAll': 'rgb(0, 138, 64)', 'prePPI': 'rgb(96, 191, 235)', 'PTMsigDB': 'rgb(14, 130, 201)',
                      'STRING': 'rgb(58, 50, 168)', 'STRING.bind': 'rgb(158, 50, 168)'}
    k_libs_means = {'STRING.bind': [0] * 10, 'ChengPPI': [0] * 10, 'PhosDAll': [0] * 10, 'BioGRID': [0] * 10,
                    'HIPPIE': [0] * 10, 'ChengKSIN': [0] * 10,
                    'STRING': [0] * 10, 'MINT': [0] * 10, 'mentha': [0] * 10, 'prePPI': [0] * 10,
                    'PTMsigDB': [0] * 10}

    results_mr_t = [r for r in results['Integrated--meanRank'] if len(r['Library'].split(';')) >= threshold][:10]
    results_mr_t.sort(key=lambda kin: sum([int(k.split(',')[1]) for k in kin['Library'].split(';')]))
    sorted_kinases = [k['TF'] for k in results_mr_t[:10]]

    for i, kin in enumerate(results_mr_t[:10]):
        scores = kin['Library'].split(';')
        for s in scores:
            l, s = s.split(',')
            k_libs_means[l][i] = int(s)

    fig = go.Figure(data=[go.Bar(name=k_lib, x=k_libs_means[k_lib], y=sorted_kinases,
                                 marker=go.bar.Marker(color=k_libs_palette[k_lib]), orientation='h') for k_lib in
                          sorted(k_libs_means)])

    fig.update_layout(barmode='stack')
    fig.update_layout(title={
        'text': 'Stacked Bar Chart of Sum of Ranks in Different Libraries',
        'y': 0.87,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
    },
        xaxis_title='Sum of Ranks in Different Libraries',
        yaxis_title='Kinases',
        font=dict(
            size=16,
            color='black'
        )
    )

    display(HTML(
        '<h2>This interactive bar chart summarizes the scaled rankings of the top kinases from the different libraries.</h2>'))
    display(HTML(
        '<i>All rankings were scaled so that each library was weighted equally. Each library contributed at most 1 (last ranking) and at least 0 (#1 ranking). ' +
        ' Any kinases without rankings for a certain library were given a ranking of 0.5 for that library. ' +
        'To download the file as a PNG, click the camera button in the upper right corner of the chart.</i>'))
    fig.show()

    # Create dictionary of dictionaries for the top 10 kinases for each ranking type
    # Format: rankings['Rank Type'] = {1: Kinase, 2: Kinase, 3: Kinase, ...}
    rankings = {}

    for library in range(0, 13):
        rankings[list(results.keys())[library]] = {}
        for rank in range(1, num_kinases + 1):
            rankings[list(results.keys())[library]].update({rank: list(results.values())[library][rank - 1]['TF']})

    # Format: ranks[0] will contain the #1 kinase for the ranking type
    for library in range(0, 13):
        ranks = []
        for rank_num in range(0, num_kinases):
            ranks.append([rank_num + 1])
        for rank in range(1, num_kinases + 1):
            ranks[rank - 1].append(list(rankings[list(results.keys())[library]].values())[rank - 1])

    display(HTML('<h2>The following tables display the integrated (summarizing) results of the top kinases.</h2>'))
    for library in range(0, 2):
        display(HTML(tabulate(ranks, headers=[list(rankings.keys())[library]], tablefmt='html')))

    display(HTML('<h2>The following tables display the rankings of the top kinases from each library.</h2>'))
    for library in range(2, 13):
        display(HTML(tabulate(ranks, headers=[list(rankings.keys())[library]], tablefmt='html')))      